<a href="https://colab.research.google.com/github/Carllllllll/Proposal/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 형태소 분석기 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [99]:
import os
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

os.getcwd()
os.chdir('/content')

In [100]:
df1 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2019_1.xlsx', index_col=None, header=0)
df2 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2019_2.xlsx', index_col=None, header=0)
# df3 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2020_1.xlsx', index_col=None, header=1)
df4 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2020_2.xlsx', index_col=None, header=0)
df5 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2021_1.xlsx', index_col=None, header=0)
df6 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2021_2.xlsx', index_col=None, header=0)
df7 = pd.read_excel(os.getcwd() + '/drive/MyDrive/data/2022_1.xlsx', index_col=None, header=0)

In [101]:
# df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)
df = pd.concat([df1, df2, df4, df5, df6, df7], ignore_index=True)

In [93]:
# list(df)
df['제안종류'].drop_duplicates()

0    타부서제안
Name: 제안종류, dtype: object

In [102]:
df = df.loc[df['제안종류'] == '타부서제안']

In [103]:
def remove_basic(tmp) :
  tmp = tmp.replace('1. 제안배경', '')
  tmp = tmp.replace('* 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성 ', '')

  tmp = tmp.replace('2. 개선안(제안내용)', '')
  tmp = tmp.replace('* 제안내용의 개선방안 또는 개선가능 방향에 대해 작성 ', '')

  tmp = tmp.replace('3. 예상효과', '')
  tmp = tmp.replace('※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.', '')
  tmp = tmp.replace('(1) 재무 성과 (年 기준/천원 단위)', '')
  tmp = tmp.replace('(2) 업무 효율화 (年 기준)', '')
  tmp = tmp.replace('(3) 기타 효과', '')

  tmp = tmp.replace('   ', '')
  tmp = tmp.replace('  ', '')
  
  tmp = tmp.replace('* ', '')
  tmp = tmp.replace('*', '')
  return tmp

In [104]:
remove_basic(tmp)

'신용카드 등록신청 업무가 영업파트인 지점에도 업무가 이관되면서, 플래너 및 고객들이 직접 신용카드 등록 변경을 요청하는 경우가 많은데.. 특히월말같은 경우 플래너들께서 습관적으로 입금보관을 하고 카드 승인을 하는데.. 예를들어 계속분 5건중 1건이라도 미등록된 카드가 있으면 입금보관 을 해제하고 다시 카드등록을 하고, 입금보관을 눌러야하는 번거로운 상황이 발생. 신용카드 등록신청 화면에서 등록요청시 입금보관이 되있을경우 등록 불가하다는 팝업창이 뜨는데.. 바로 그 안에서 해결 할 수 있게 화면내 방문수금확정 화면을 바로 갈 수 있는 key를 만들어 주면 플래너 본인이 직접 해제를 함으로써 총무에게 일일히 입금보관 취소해달라 말도안해도되고 총무 또한 복수 업무를 계속적으로 반복하지 않아 업무 시간도 절약될 뿐 아니라, 전산을 사용함에 있어서도 편리 할 것으로 보입니다.(요청내용 파일 첨부완료) '

In [108]:
df['제안내용'] = df['제안내용'].astype(str).apply(remove_basic)

TypeError: ignored

In [56]:
df.dropna

<bound method DataFrame.dropna of                                       제목  \
0                       장기 이재력 플래너 열람 가능   
1      신용카드 등록 화면, 방문수금 확정 바로가기 key 생성요청   
2                   직불치료비 지급처명 조회하기 기능신설   
3                       위촉증명서발행시코드번호자동입력   
4                          메신저 알리미 팝업 설정   
...                                  ...   
18161                   상품설명서 전산 인식 개선요청   
18162                    이재 중 계약해지 절차 개선   
18163                집금책임명세 전산 화면 개선 요청    
18164                 트레일러 설계시 견인차량번호 확인   
18165  자동차보험 자동차번호 및 차대번호는 확인되도록 변경해주세요.   

                                                    제안내용  
0      장기 이재력의 경우 플래너는 개인정보 보호로 인해 열람이 제한 되어 있음 그러나 확...  
1      신용카드 등록신청 업무가 영업파트인 지점에도 업무가 이관되면서, 플래너 및 고객들이...  
2      현재 대인 치료관정관리화면상의 직불치료비 지급에서 지급처 코드로만 미등록사항을 확인...  
3      플래너 재직증명서 출력시 해당 플래너를 조회하고 재직증명서 발행 버튼을 클릭하면 해...  
4      매주 월요일 "관리계약 회신요청알림"이 팝업으로 뜨고있습니다. 몇 십건의 팝업이 발...  
...                                                  ...  
18161  상품설명서 필수스캔이 되면서, 

In [57]:
# mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
mecab = Mecab()
tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    
    text = row['제안내용']
    tag = row['제목']
    if pd.isna(text): continue
    if pd.isna(tag): continue
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))
    print(tag)

print('문서의 수 :', len(tagged_corpus_list))

  1%|          | 174/18166 [00:00<00:20, 898.89it/s]

장기 이재력 플래너 열람 가능
신용카드 등록 화면, 방문수금 확정 바로가기 key 생성요청
직불치료비 지급처명 조회하기 기능신설
위촉증명서발행시코드번호자동입력
메신저 알리미 팝업 설정
미납해지 조회하면에서 플래너별 조회후 발행키가 있었으면 좋겠습니다.
일반화재보험 설계시 층별업종을 입력하면 높은요율을 표시해주셨음 합니다.
고심도센터 운영
타사 피구상   추가접수
하이플래너 가상계좌 본인 발송 활성화 요청
나눔특약 심사 화면 개선 요청
기타환급금 10개 신청 불가 에러 해소요청
집금 보관후 카드건 일괄승인시 다른카드건이 있다 하여 승인이 안됨
만기환급금 계약관리 안내장 점포에서 재발송 신청할수 있도록 조치해주세요
자동차청약서 기타손해보험계약 다수가입 가입자동의서 스캔관련
실효 후 감액 부활 시 휴면상태에서도 부활 가능
장기계속분선납일괄처리
자동차 전자서명추후보완
환급금 지급 화면 개선 요청
출산지원금 대체납입선납 화면 개선
화재특종 의무보험 만기건 담당 플래너 문자전송
보험계약대출 지급입력시 수령권자 구분값 개선
납세사실증명원 인쇄시 재직기간오류
보안메일 발송시스템 개선
신용카드 수납등록건 결재일자별 화면 구성 요청
0원 산출건 심사자 자동배당 프로세스 개발
영업포탈-e모니터링안내문자 빨간색 진한 테두리 표시좀 해주세요!!!
스캔팩스 메신저연동
가상계좌번호 부여 &gt; 알림톡전송 &gt; 입금보험료 자동세팅
이재사항 확인업무 - 총무확인 =&gt; 플래너확인으로 변경요청합니다
진위여부 확인 및 발급 관련 개선 요청
직배당 접수건중 일부면책후 추가계약 등록 제한으로 전산개선요청
하이라이프와 외부손사법인 업무 이관 지침 개선
피해자 조회 시 옵션
질병 /상해 와 배상 착오 접수건의 사고접수 호환 요청
하이포탈 장기 정보입력 화면 관련 제안
주차장 시설 소유 관리자 보험 한도 표시 
관리계약 점포관련 서류 세밀화 
콜센터 치매보험 문의시 인수점포에 전화 연결되도록 프로세스 보완 필요 
하이포탈 장기 자동산출 화면 관련 제안
피해자물 등록화면에서 다수피해자명 피해자로

  2%|▏         | 351/18166 [00:00<00:20, 851.62it/s]

임직원봉사활동중 하이갓탤런트 별도운영
보험금청구서 인쇄물요청
장애아동 대상 신규 사회공헌프로그램 실시
교통사고사실확인원 여러번호의 팩스로 일괄 전송기능 추가요청
새로운 업무 환경에 따른 평가 방법 개선
사회공헌 프로그램간 연계활동을 통한 업무효율화
여행자보험 인원수로 설계건 청약서 발행제한
사고현장출동자 면허증 확인.
담당자 팀 부서 평가 일치를 통한 손해액 관리
장기 신계약 담보누적 자율조정 관련 제안드립니다.
세만기 담보 완납 계약의 갱신보험료 별도 표기 요청
보상 접수번호조회화면 하단 탭(링크) 동일화
합의금 산출 치료비 입력
HI-UP 사진 등록 기능 개선
하이미 로그인 한번으로 모든 시스템(e-hr,하이포털 등) 이용
합의관리 입력 시 지불보증중지와의 연동 중지
구상금 자동 반제 및 환입 시스템
종결시 지불중지서 미발송건 자동발송
조치기한 안내 통한 누락 방지
HI-UP 관련 지불 보증 발송
실손단독형 보험 12회차 단위 선납 약관 개정 요청
마음봇 이모티콘 생성
자동이체 신청경로 유선(녹취)선택시 증빙정보 입력란 팝업순서 변경요청
음식물배상책임담보Ⅱ 신설
배서로 추가가 안되는 특약 표기요청
다이렉트홈페이지)갱신보험료 예측에 대한 메뉴 신설 (타사 적용 중)
다이렉트홈페이지) 보험료산출근거 상세발송 및 pc 만기알림 설정요청
일반청약입력시 이미지팝업창 오픈신설
재직증명서 사업자번호 출력 요청 
신용카드승인화면에  90일경과후 취소시 카드사수수료미환급표기요청
일반 전자서명 진행 사항 확인
치아보험 장기사고접수시 화면개선(계약종목 선택 비활성화)
자동차 수납전 확정 건 보험료 입금표 자동연결
신용카드 등록시 플래너 담당계약 아닌 경우 비활성화 요청.
휴대폰서명 즉시이체시 확정요청까지 업무 요청
계약변경처리중 보상접수 불가 멘트 요청
일반보험 전자서명 관련 전산 개선요청
동일증권 멘트및 변경요청
지급보증정보 중계시스템 및 ARS  지급보증시 입원여부 체크하기
SAP 총괄 임차현황 내 임차료 VAT포함 금액 조회 기능 추가
fg 경과도과건 이첩 예정기능 신설

  4%|▎         | 637/18166 [00:00<00:18, 928.22it/s]

...

자동차 요율심사 화면 수정요청 
납입증명서 발행업무 개인정보 마스킹 관련 개선 요청
[미수요청]시 중복 서류 생략 요청
갱신후보험료 알림톡 발송 화면 개선 요청
심사관리플래너 세부내용 확인
입금취소 관련 제안
단체실손&#62;개인실손 전환(2종) 
대장(결장)점막내암 자동산출 관련
장기 계약변경 결재 화면 개선 요청(요청취소 버튼 추가)
계약변경 프로세스 개선관련 추가보완의건 
장기 휴대폰전자서명시 이메일 입력란에 이메일/핸드폰번호 중 선택할수 있도록 변경 요청
일반보험 발행 업무 개선요청
전기차 보험에 긴급 충전 서비스 특약 상품 개발
사랑드림 유형별 가입예시표 발행요청
해촉요청자(전자서명) 진행관리화면 별도 생성  또는 알림톡 요청
CM상품 설계 및 가입을 위한 실명인증 기준 재정비
신용카드 등록신청  화면 개선요청
설계명세 화면에서 착한 실손 설계건만 조회할 수 있도록해주세요.
서베이 보고서에 대한 손해사정 검토 의견 입력시, 서베이측에 문자 전송요청
점포장/직원 SMS 발송 시 이름검색기능 추가 요청
유사암진단시 (기타피부암,갑상선암,제자리암) 각각 1회 보상 관련
사고상세조회 화면 관련 개선 제안
손해사정 진행요청서 기능 추가
계약자변경 양수양도계약서 서식
완전판매모니터링 피보험자 모니터링 채널 개선
s포탈 계약자 법인 단기운전자 개선요청
개인정보 오류 입력으로 인한 고객민원관련 - 개인정보 입력자 조회화면 개선요청 ( GA설계 매니저 의욕 저하 개선)
계약자 본인外신용카드 수납신청서 휴대폰서명 및 증빙서류 파일첨부 개발제안
점포장 여비교통비의 일괄지급 요청
계약자변경서류 대출약정서 별도 발행 개선 요청
신계약 자동이체 등록건별명세에 신용카드 등록여부 표기
대리점 내부이관 관련
신용카드 등록신청 화면 내 "추가"버튼 생성 요청
자동갱신 거절/취소에 대한 모바일 서명 진행 
계약이관 결재 버튼 위치 변경 요청
계약체결전 완전판매모니터링 알림 개선요청
질의회신명세 조회기준 기본값 변경
차수 관계없이 완전판매확인서 스캔시 점검종료되도록 조치
타카드 신용카드

 99%|█████████▉| 18037/18166 [00:28<00:00, 635.03it/s]

약관 내용 오기재 수정 요청
지급 취소(콜센터지급건) 전산에서 가능하도록 구현요청 
장기보험 확정요청시 계약자 / 피보험자 만나이 확인가능하도록 전산구현 요청
완전판매 화면 개선
자동차 추가부속심사시 증빙서류 안내팝업창 생성요청 
신계약 청약 후 증권 발행
하이바로승인건 심사 재요청시 심사구분 선택가능하도록 개선요청
이미지파일  가상팩스 전달 화면 구현요청  
총총이 2~4회차 미집금 리스트 이미지 수정
담보해지변경(복구)취소 처리화면 감액도 가능하게끔 변경요청
영업포탈에서 이관계약 리스트 엑셀내려받기 가능하게 전산 개선 요청
체중계 발송조회 화면에서 송장번호 바로 확인가능하도록 전산구현 요청
부당청구조정관리 화면 에러
환급금 조회 화면에서 바로 지급 요청 버튼 생성 요청
일반 아파트단체보험 필수서류등록시(심사요청관련)
플래너 신계약 자동확정 권한 제한요청
전자서명시 계약번호 전체 보이게 변경해주세요
암진단,항암방사선약물치료 지급시 지급결의서에 특약소멸 처리 되었으나 계약사항 특약 존재함.
자동차보험 캠핑카할인할증 적용관련 전산개선요청 
행복을다모은보험 상품 계약대출 진행시 세부내용 안내요청 및 팝업
장기사고접수후 접수취소 권한 및 화면개설 요청
 [참석여부 입력] 화면 일괄선택 체크 요청
질의회신 결과 edms 등록
일반 단체보험 청구시 다른 계약번호 같이 체크하여 접수 할 수 있도록 요청
동계 자동차 손해율 감소를 위한 월동장비 할인 특약 추가 요청
철회 긴급 요청건
RPA활용 대외민원 금감원 처리결과 자동화
유의계약 명세 개선요청
계약변경 카톡알림 개선
일반보험 청약서 발행 개선 
FG-&#62;일반보상부 발령자를 위한 중상자 조사 기법 교육자료 제작
장기보험 약관 신청 개선 요청
계정명세조회 합계금액 표기
배양등록 이중 입력 제한 요청합니다.
신인활동성과 화면 엑셀다운로드 개선요청
선지급보증 체결 동의 여부 화면 개선요청
만기시 마일리지 자동환급 추가 알림톡 요청
신용카드 전자서명시 필수서류가 미스캔일때 확인 할 수 있도록 제안
경리단일입금 화면 신용카

100%|██████████| 18166/18166 [00:28<00:00, 631.20it/s]


일반보험 휴대폰 전자서명 쪽지 수정요청
계약자+피보험자 전자서명시 피보험자 완전판매모니터링 불가
단순설계건 S포탈에서 보장내역 조회가능하게 요청
신용카드 등록 모바일서명 발송 관련 제안
알림톡 발송된 고객명, 계약번호 마스킹처리 개선
연말정산대상조회 
E모니터링 안내문자 발송 시 다수 피보험자 선택 요청
포탈내 주소등록 프로세스 안내 수정요청
납입최고 재전송 화면 개선 요청 
일괄변경 EDMS팩스 등록 요청
보험료 갱신설계시 연동되는 전보험 마일리지 정산 화면 개선 요청 드립니다.
납입최고 전자문서 발송 화면 건의
신분증 진위여부 확인 화면 플래너 전산에서도 오픈 되도록 제안
전자서명다건일경우한번에인증
지급제한 해제 전산화면 개선요청
손해액절감과 생산성을 동시에 반영한 '발생손해액 절감액 ' 평가방법 도입
완전판매 점검종료건 관련 제안
계약자 ≠ 예금주 자동이체 등록시 진위여부 동시진행
영업지원자료실에 자료확인좀 해주세요
선임총무 전산 계좌번호 입력 불가능 개선 요청 
영업포탈 화면 개선요청 (TC지점 ITP통화시 화면)
업무별처리현황 통계_지사코드 로그인시 화면개선요청
공인인증 전자서명하기 어려워요(계약체결/계약관리)
 E모니터링 화면 수정요청
법인고객 등록 플래너 입력 권한 확대 요청 
마일리지특약Ⅰ-&#62;마일리지특약Ⅱ로 전환 가능하도록 전산 개선요청
실손전환 원스탑 설계화면 전산개선
환급금 조회 화면 계좌번호 수정 가능 요청 
신용카드 등록 불가 개선 요청 
자동차 증권 발행 프린터 선택 개선 요청 
적용요율 등급확인서 차량번호 마스킹처리 해제요청
사진전송시 카카오톡으로 전송가능 요청
주택화재 필수서류 서식 등록요청
하이프린트 승인접수건 알림쪽지 개선요청 
전자청약 자동확정 예외조건 적용요청 
보고서 진행의견 입력 시 자동문자전송 기능 추가
경위서 작성 간편화를 위한 전산개발 요청
자동차 설계건 조회시 화면 수정 요청 드립니다.
다자녀 계약 전자서명 건 청약서 커버페이지 자녀수 기재요청
모니터링 미진행 화면 개선 요청
휴대폰 전자서명시 
모바일 전자

In [59]:
tagged_corpus_list[0]

TaggedDocument(words=['장기', '이', '재력', '의', '경우', '플래너', '는', '개인', '정보', '보호', '로', '인해', '열람', '이', '제한', '되', '어', '있', '음', '그러나', '확인', '하', '고자', '한다면', ',', '총무', '를', '통해', '이', '재력', '을', '확인', '하', '고', '있', '어', '얼마', '든지', '열람', '이', '가능', '한', '상태', '임', '이러', '한', '과정', '은', '플래너', '총무', '를', '거친', '이중', '작업', '임', '플래너', '또한', '장기', '이', '재력', '열람', '이', '가능', '하', '고자', '전산', '변경', '요청', '하', '는', '바', '임', '(', '단', ',', '열람', '시', '플래너', '사', '번', ',', '비밀', '번호', ',', '열람', '하', '고자', '하', '는', '사유', '기재', ',', '필요시', '고객', '동의', '인증', '까지', ')', '총무', '를', '거친', '이중', '업무', '해소', '플래너', '불만', '감소'], tags=['장기 이재력 플래너 열람 가능'])

In [60]:
from gensim.models import doc2vec

In [61]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
# print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('/content/drive/MyDrive/Proposal/dart.doc2vec')

In [62]:
import os
os.getcwd()
os.chdir('/content/drive/MyDrive/Proposal/')

In [63]:
df[40:43]

,제목,제안내용
40,피해자물 등록화면에서 다수피해자명 피해자로 입력가능하도록 개선요청,"장기손사 업무중 피해자물 등록시 다수의 피해자경우, 일부 합의종결하고 가지급할경우 ..."
41,고객안내(SMS/LMS/알림톡) 재발송 기능 도입,nan
42,매달 둘째주 FG 업무 쉐어링 관련,매달 둘째주 월요일 FG센터의 업무 분산효과와 초동안내의 신속조치로 유선센터의 업무...


In [64]:
df.at[1, '제목']

'신용카드 등록 화면, 방문수금 확정 바로가기 key 생성요청'

In [65]:
# df.iat[i, 1]
i = 5
str = df.at[i, '제목']
print(str)

미납해지 조회하면에서 플래너별 조회후 발행키가 있었으면 좋겠습니다.


In [66]:
similar_doc = model.docvecs.most_similar(str)
print(similar_doc)

[('의료기관별 합의참고정보 활용 고도화 방안', 0.46053168177604675), ('구실손 관리계약리스트 내 발행버튼 생성 요청', 0.42610716819763184), ('미납해지 리스트 발행버튼 추가요청', 0.4195152223110199), ("보상포탈 화면 '일정'에서 접수번호를 클릭하면 바로 '업무처리' 화면으로 갈 수 있도록 링크 개선 부탁드립니다. ", 0.41605597734451294), ('업적추진비 - 총괄표 플래너별 지급현황을 일괄 발행 가능하도록 요청', 0.4134369492530823), ('정액담보 당타사 초과계약현황 발행버튼 생성요청 ', 0.40470942854881287), ('고객연락처 전화번호_핸드폰번호 입력시 사전 인증으로 오류입력 방지', 0.39861926436424255), ('스캔문서 출력', 0.396512895822525), ('영업포탈 완전판매조치대상에서 계약조회가능토록 개선요청', 0.3876953721046448), ('장기 미발행 증권 출력 제한 삭제 요청 ', 0.38696128129959106)]


In [67]:
str2 = similar_doc[1][0]

In [68]:
df.loc[df['제목'] == str].iat[0,1]

'월 마감후 미납해지 계약을 플래너분들에게 서면으로 나눠주고 있는데, 발행키가 없어서 엑셀내려받기후 편집해서 나눠주고 있습니다.플래너별 조회후 발행키가 있어서 깔끔하게 출력되었음 합니다.플래너분들이 스스로 미납해지 계약을 조회하고 안내하는 경우가 별로 없어총무들이 엑셀내려받기후 플래너분들에게 나눠주고 안내후 부활하고자 하는데업무의 편의성이 증대될것 같습니다.'

In [69]:
df.loc[df['제목'] == str2].iat[0,1]

'구실손 관리를 위해 구실손 관리계약리스트를 활용.플래너 개인별 리스트를 본인이 출력하길 원함.엑셀 내려받은 후 발행을 유도하였으나, 엑셀 정리를 어려워하심.구실손 관리계약리스트 내에 발행버튼이 생성되었으면 합니다.플래너 개별적으로 효율적인 구실손 관리를 할 수 있음. ※예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'

In [70]:
similar_doc[1][0]

'구실손 관리계약리스트 내 발행버튼 생성 요청'

In [71]:
# 정확도 높은 건들 찾기

for i in range(1,len(df)):
    
    try:
        str = df.at[i, '제목']
        similar_doc = model.docvecs.most_similar(str)
    except:
        a = 3

    if similar_doc[1][1] > 0.8:
        print(i)
        print(str)
        print(similar_doc[1][0])

4268
생체인증 결재시스템 오픈을 통한 보상업무 효율화
사옥관리 일상업무 전산화
4330
치료비 자동심사 지급건 보상 종결건 및 추산 부족건 심사불가 해결
자동차보험 전체해지시 신용카드 부분취소 프로세스 도입 
4342
자가용 이용 '쿠팡 플렉스' 사고 자동차보상 면부책 검토
보험조사 혐의자사항 수정이력 확인 전산개발
4347
소송 판결내용 알림서비스 수신대상 확대(CS담당자 추가)
보험조사분석사(CIFI) 자격취득 활성화를 위한 지원제도 개선
4351
소송종결 화면(소송비용 항목)개선을 통한 업무 효율화 및 구상누락 방지
자동세차기 소송대응 방법
4375
하이유피 개선 안내
보험금청구 필수구비서류 (청구서, 개인(신용)정보처리동의서) 변경사항 전산 반영
4405
경비업자특약의 공통특약 전산누락 수정
말초신경장해 판결경향 보고
4408
마일리지특약 문자개선을 통한 민원예방 및 비용절감
영업용 택배차 자동요율산출 개발
4412
스크래핑을 통한 부속품 입력 및 부속연계 할인형특약 가입프로세스 개선
영조물 공학감정 분석 사례를 통한 구상총량증대 방안
4413
마일리지 특약 플래너 보완요청 알림톡 신설을 통한 총무업무 효율화
척추골절장해 신체감정서 분석 및 대응방안 마련
4416
마일리지특약 보완요청업무 간소화를 통한 콜센터 업무효율 개선
소액소송 응소기준 변경으로 인한 전산 개선
4417
법인건 커넥티드카 전문 자동조회를 통한 특약 가입건수 확대
보험조사 혐의자사항 수정이력 확인 전산개발
4418
맞춤형 특약 발송요건 개선
카카오 알림톡 자동차보상 채널 신설
4419
자동차보험 커넥티드 할인특약 전문조회 대상 확대
상호반제위한 지급결의 매뉴얼(타서열이관환입)
4421
자동차보험 차명코드 조회화면 개선
사택지원대상 자동안내 시스템 구축
4423
자동차보험 증명서 발행프로세스 개선
모바일 한방 치료확인서 시스템 구축
4424
자동차보험 모바일 증권/약관 발송 활성화
소송관리 화면 내 관련담보 및 쟁점 별 검색 기능 추가 
4446
자동차보험 단기운전자확대특약 가입방식

KeyboardInterrupt: ignored

In [ ]:
df.at[2250, '제안내용']

'1. 제안배경         * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성           매일 조치대상에서 완전판매조치 안내 및 처리를 하고있습니다.     오전/오후 나누어 보고있는데     종종 처리할수없는 (해외, 야간일을하여 오전모니터링 불가등) 건들을 플래너에게 전달받았음에도 불구하고     다음날 또 처리가 안되어 이중으로 연락을 하게됩니다.     또한 후임총무와 함께 업무처리를 하게되면 한 플래너에게 이중연락등 중복되게됩니다.     2. 개선안(제안내용)        * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성            하단에 메모기능을 추가하여     메모가 되어있는부분은 처리전 한번더  볼수있도록 제안합니다.           3. 예상효과        ※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.           (3) 기타 효과           시간감소, 효율적 업무처리 가능        ※  예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'

In [ ]:
str = df.at[2250, '제목']
similar_doc = model.docvecs.most_similar(str)

In [ ]:
similar_doc

[('조치대상 화면 보완', 0.847792387008667),
 ('조치대상 화면 제안', 0.8416188955307007),
 ('대리점 조치대상화면 수정요청합니다.', 0.43335217237472534),
 ('콜적부 조치기한 알람 뜨도록 개선요청', 0.41208720207214355),
 ('플래너 본인 계약 시 모집자 서명 안해도 확정가능하도록 제안합니다. ', 0.39692389965057373),
 ('완전판매 모니터링 최종차수 조치내용 관련', 0.39235469698905945),
 ('플래너 본인계약 모집자서명 생략', 0.38509178161621094),
 ('완전판매 모니터링 대상 리스트 제공필요', 0.375488817691803),
 ('하이포탈에서 어린이보험 설계건 조회 요청', 0.3726857304573059),
 ('장기 계속보험료 입금 취소건 지점 전결 처리', 0.3718041181564331)]

In [ ]:
df.loc[df['제목'] == '조치대상 화면 보완'].iat[0,1]

'1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성     매일 조치대상에서 완전판매조치 안내 및 처리를 하고있습니다.  오전/오후 나누어 보고있는데  종종 처리할수없는 (해외, 야간일을하여 오전모니터링 불가등) 건들을 플래너에게 전달받았음에도 불구하고  다음날 또 처리가 안되어 이중으로 연락을 하게됩니다.     2. 개선안(제안내용)     * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성      하단에 메모기능을 추가하여  메모가 되어있는부분은 처리전 한번더  볼수있도록 제안합니다.     3. 예상효과     ※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.        (3) 기타 효과     시간감소, 효율적 업무처리 가능     ※  예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'

In [ ]:
df.loc[df['제목'] == '조치대상 화면 제안'].iat[0,1]

'1. 제안배경      * 제안 사항의 현황/프로세스 또는 개선대상/문제점 작성     매일 조치대상에서 완전판매조치 안내 및 처리를 하고있습니다.  오전/오후 나누어 보고있는데  종종 처리할수없는 (해외, 야간일을하여 오전모니터링 불가등) 건들을 플래너에게 전달받았음에도 불구하고  다음날 또 처리가 안되어 이중으로 연락을 하게됩니다.     2. 개선안(제안내용)     * 제안내용의 개선방안 또는 개선가능 방향에 대해 작성      하단에 메모기능을 추가하여  메모가 되어있는부분은 처리전 한번더  볼수있도록 제안합니다.     3. 예상효과     ※ 성과우수제안 선정의 주요기준이므로 상세하게 기술바랍니다.        (3) 기타 효과     시간감소, 효율적 업무처리 가능     ※  예시 활용목적으로 신분확인 서류 등의 첨부시 개인정보가 노출되지 않도록 각별히 주의바랍니다.'